In [ ]:
system_prompt = 'You are a programmer,you use python,sympy to solve problems,set variables,exprestions,set Interval to give range of variable,and use solve() function to calculation.only give a python code'

### load data

In [3]:
import json
from datasets import load_dataset, Dataset
with open('conic10k/test.json', 'r', encoding='utf-8') as file:
    raw_conic_tr = json.load(file)
datasett = load_dataset('json', data_files='conic10k/test.json', split=None)
# dataset1['train']['text']
# len(dataset1)
datasett['train'][flag]['text']

'已知$c$是双曲线$C$:$\\frac{x^{2}}{a^{2}}-\\frac{y^{2}}{b^{2}}=1,(a>0, b>0)$的半焦距，离心率为$e$，则$\\frac{1}{e}+\\frac{b}{c}$的最大值是?'

### prompt to have answer

In [4]:
#init data

ans = {'Ai_answer':[],'real_answer':[],'primary_output':[]}
flag = 500
my_dataset = datasett
print(my_dataset)

DatasetDict({
    train: Dataset({
        features: ['query_spans', 'text', 'query_expressions', 'process', 'fact_spans', 'answer_expressions', 'fact_expressions'],
        num_rows: 2069
    })
})


In [5]:
#将大模型输出的结果中的代码提取并运行，将结果返回
import re
import sys
from io import StringIO
def to_py_and_run(text):
    pattern = r'```python\n(.*?)\n```'
    match = re.search(pattern, text, re.DOTALL)
    run_ans = 'None'
    if match:
        code_block = match.group(1).strip()  # 提取代码块并去除首尾空白
        
        # 截取最后一行提取代码结果
        last_line = code_block.rstrip().rsplit('\n', 1)[-1]

        # #最后一行可能是print(answer_expr),也可能是answer_expr，我们归一化处理
        ppattern = r"print\((.*)\)" 
        pmatch = re.search(ppattern, last_line)
        pmatch
        if pmatch:
            no_use=1 # jump
        else:
            code_block = code_block + '\n\nprint('+ last_line+')'
        
        # with open('outputcode.py', 'w') as file:
        #     file.write(code_block)  # 将代码写入文件
        # #为了保存输出，使用重定向


        old_stdout = sys.stdout
        new_stdout = StringIO()
        sys.stdout = new_stdout

        # 运行生成的Python文件 保存输出到返回值，并恢复输出定向
        try:
            exec(code_block,globals())
            output = new_stdout.getvalue()

        except:
            run_ans = 'code_error'
        else:
            # print('printsuccess')
            run_ans = output.strip('\n')
        sys.stdout = old_stdout
            
    return run_ans

In [31]:
# ans['primary_output']

In [20]:
from gradio_client import Client
client = Client("Qwen/Qwen2-72B-Instruct")
i=flag
while i < len(my_dataset['train']['text']) and i<1000:
  mathproblem = my_dataset['train'][i]['text']
  problemans = my_dataset['train'][i]['answer_expressions']
  result = client.predict(
  query=mathproblem,
  history=[],
  system="You are a programmer,you use python,sympy to solve problems,set variables,set Interval to calculate the range of question,exprestions,and use solve() function to calculation.only give a python code.(code cannot have external inputs and graphical outputs)",
  api_name="/model_chat"
  )
  print('finish predict')
  ans['primary_output'].append(result[1][0][1])
  code_result = to_py_and_run(result[1][0][1])
  if code_result =='code_error':
    stop_point = 0
    stop_limit = 1
    while code_result =='code_error' and stop_point < stop_limit:
      stop_point = stop_point+1
      print('error time:',stop_point)
      new_result = client.predict(
        query='you gave me a code to solve a problem,code:'+code_result+'problem:'+mathproblem+'pleas debug the code,then only give me a corrected python code',
        history=[],
        system="You are a programmer,you use python,sympy to solve problems,set variables,set Interval to calculate the range of question,exprestions,and use solve() function to calculation.only give a python code.(code cannot have external inputs and graphical outputs)",
        api_name="/model_chat"
      )
      code_result = to_py_and_run(new_result[1][0][1])

  ans['Ai_answer'].append(code_result)
  ans['real_answer'].append(problemans)
  print(i)
  print(mathproblem)
  print(code_result)
  print(problemans)
  with open('test_predict_2.json', 'w', encoding='utf-8') as file:
    json.dump(ans, file)
  i = i+1
  flag = i

Loaded as API: https://qwen-qwen2-72b-instruct.hf.space ✔
finish predict
940
过双曲线$\frac{x^{2}}{a^{2}}-\frac{y^{2}}{b^{2}}=1(a>0, b>0)$的一个焦点$F$引它的渐近线的垂线，垂足为$M$，延长$FM$交$y$轴于点$E$，若$F M=M E$，则双曲线的离心率为?
-sqrt(3)
sqrt(2)
finish predict
941
双曲线的中心在原点，焦点在$y$轴上，焦距为$16$，一条渐近线方程为$y=\frac{3}{\sqrt{7}} x$，则双曲线方程为?
Eq(-x**2/28 + y**2/36, 1)
y^2/36-x^2/28=1
finish predict
942
$P$为椭圆$\frac{x^{2}}{25}+\frac{y^{2}}{9}=1$上一点，$F_{1}$、$F_{2}$为左右焦点，若$\angle F_{1} P F_{2}=60^{\circ}$，则$\Delta F_{1} P F_{2}$的面积为?
27.7128129211020
3*sqrt(3)
finish predict
943
已知椭圆$\frac{x^{2}}{5}+\frac{y^{2}}{4}=1$的左焦点为$F_{1}$，右焦点为$F_{2}$，过$F_{1}$作$x$轴的垂线与椭圆相交于$A$、$B$两点，则$\triangle A B F_{2}$的面积为?
1.6*sqrt(5)
8*sqrt(5)/5
finish predict
944
已知椭圆$\frac{x^{2}}{a^{2}}+\frac{y^{2}}{b^{2}}=1(a>b>0)$的右焦点为$F$，过$F$点作$x$轴的垂线交椭圆于$A$、$B$两点，若$\overrightarrow{O A} \cdot \overrightarrow{O B}=0$，则椭圆的离心率等于?
-b*sqrt(-1/((a - b)*(a + b)))
(-1+sqrt(5))/2
finish predict
945
双曲线$C$: $\frac{x^{2}}{a^{2}}-\frac{y^{2}}{b^{2}}=1(a>0, b>0)$, $P$为双曲线$C$上

In [16]:
# query flag
flag

898

In [26]:
len(ans['Ai_answer'])
# len(ans['primary_output'])
len(ans['real_answer'])

501

In [25]:
ans['real_answer'].append('1')
ans['Ai_answer'].append('1')

In [28]:
import pandas as pd
# 需要库：openpyxl
# 将字典转换为pandas DataFrame
memory_df = pd.DataFrame(ans)
 
# 将DataFrame写入Excel文件
# !!!!记得修改你的文件名
memory_df.to_excel('qwen2_codeversion_2.xlsx', index=1)